In [ ]:
import json, glob, os
import time
import subprocess
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable, SessionExpired, TransientError, ClientError

# ========= CONFIG =========
FOLDER_JSONS = r"TRATADOS\\*.json"
NEO4J_URI  = ""
NEO4J_USER = ""
NEO4J_PASS = ""
NEO4J_DB   = ""
SPATIAL_LAYER = "municipios_wkt"
PROGRESS_FILE = ""


SAFE_CHUNK_SIZE = 2000       
MAX_FAST_RECORDS = 120_000    


class Neo4jDatabaseManager:
    def __init__(self, uri, user, password, database):
        self.uri = uri
        self.user = user
        self.password = password
        self.database = database
        self.driver = None
    
    def ensure_database_exists(self):
        
        try:
            self.driver = GraphDatabase.driver(
                self.uri, 
                auth=(self.user, self.password),
                max_connection_lifetime=3600
            )
            
            with self.driver.session(database="system") as session:
                result = session.run("SHOW DATABASES")
                databases = [record["name"] for record in result]
                
                if self.database in databases:
                    
                    status_result = session.run(
                        "SHOW DATABASE $db YIELD name, currentStatus",
                        db=self.database
                    )
                    status_info = status_result.single()
                    if status_info:
                        status = status_info["currentStatus"]

                        
                        if status == "online":
                            return True
                        else:
                            return self._start_database()
                    else:
                        return self._create_database()
                else:
                    return self._create_database()
                    
        except Exception as e:
            return self._fallback_creation()
    
    def _create_database(self):
        try:
            with self.driver.session(database="system") as session:
                session.run(f"CREATE DATABASE `{self.database}`")
                return self._wait_for_database_online()
        except Exception as e:
            return self._fallback_creation()
    
    def _start_database(self):
        try:
            with self.driver.session(database="system") as session:
                session.run(f"START DATABASE `{self.database}`")
                return self._wait_for_database_online()
        except Exception as e:
            return False
    
    def _wait_for_database_online(self, max_wait=60):
        start_time = time.time()
        
        while time.time() - start_time < max_wait:
            try:
                with self.driver.session(database="system") as session:
                    result = session.run(
                        "SHOW DATABASE $db YIELD name, currentStatus",
                        db=self.database
                    )
                    status_info = result.single()
                    
                    if status_info and status_info["currentStatus"] == "online":
                        return True
                    else:
                        status = status_info["currentStatus"] if status_info else "unknown"
                        time.sleep(5)
            except Exception as e:
                time.sleep(5)
        
        return False
    
    def _fallback_creation(self):
        try:
            result = subprocess.run(
                [
                    "neo4j-admin", "database", "create", self.database,
                    "--verbose"
                ],
                capture_output=True,
                text=True,
                timeout=30
            )
            
            if result.returncode == 0:
                subprocess.run(["neo4j", "restart"], timeout=60)
                time.sleep(10)
                return self._wait_for_database_online()
            else:
                return False
                
        except Exception as e:
            return False
    
    def test_connection(self):
        try:
            with self.driver.session(database=self.database) as session:
                result = session.run("RETURN 'Conexão OK' as status")
                return result.single()["status"] == "Conexão OK"
        except Exception as e:
            return False
    
    def close(self):
        if self.driver:
            self.driver.close()


class Neo4jConnectionManager:
    def __init__(self, uri, user, password, database):
        self.uri = uri
        self.user = user
        self.password = password
        self.database = database
        self.driver = None
        self._connect()
    
    def _connect(self):
        if self.driver:
            self.driver.close()
        self.driver = GraphDatabase.driver(
            self.uri, 
            auth=(self.user, self.password),
            max_connection_lifetime=3600,
            max_connection_pool_size=10,
            connection_acquisition_timeout=300,
            connection_timeout=300,
        )
        with self.driver.session(database=self.database) as session:
            session.run("RETURN 1")
    
    def get_session(self):
        try:
            return self.driver.session(database=self.database)
        except (ServiceUnavailable, SessionExpired, TransientError):
            self._connect()
            return self.driver.session(database=self.database)
    
    def close(self):
        if self.driver:
            self.driver.close()

CYPHER_CONSTRAINTS = [
    """CREATE CONSTRAINT municipio_ibge IF NOT EXISTS
       FOR (m:Municipio) REQUIRE m.ibge_id IS UNIQUE""",
    """CREATE CONSTRAINT dia_key IF NOT EXISTS
       FOR (d:Dia) REQUIRE d.date IS UNIQUE""",
    """CREATE CONSTRAINT meteo_key IF NOT EXISTS
       FOR (mm:Meteo) REQUIRE (mm.municipio_id, mm.date) IS UNIQUE""",
    """CREATE CONSTRAINT casos_key IF NOT EXISTS
       FOR (c:Casos) REQUIRE (c.municipio_id, c.date, c.agravo) IS UNIQUE""",
    """CREATE CONSTRAINT notif_key IF NOT EXISTS
       FOR (n:Notificacao) REQUIRE n.notif_id IS UNIQUE""",
    """CREATE CONSTRAINT tipo_ativ_key IF NOT EXISTS
       FOR (t:TipoAtividade) REQUIRE t.codigo IS UNIQUE""",
    """CREATE CONSTRAINT ativ_exec_key IF NOT EXISTS
       FOR (a:AtividadeExec) REQUIRE a.exec_id IS UNIQUE""",
    """CREATE CONSTRAINT agravo_codigo IF NOT EXISTS
       FOR (g:Agravo) REQUIRE g.codigo IS UNIQUE""",
    """CREATE CONSTRAINT ativ_resumo_key IF NOT EXISTS
       FOR (ar:AtividadeResumo) REQUIRE (ar.municipio_id, ar.date, ar.tipo) IS UNIQUE""",
    """CREATE INDEX ativ_exec_municipio_date_sisaweb_tipo IF NOT EXISTS
       FOR (a:AtividadeExec)
       ON (a.municipio_id, a.date, a.sisaweb_tipo)"""
]

CYPHER_UPSERT_MUNICIPIO = """
MERGE (m:Municipio {ibge_id: $ibge_id})
  ON CREATE SET
      m.nome        = $nome,
      m.sisaweb_id  = $sisaweb_id,
      m.geojson_str = $geojson_str,
      m.wkt         = $wkt
  ON MATCH SET
      m.nome        = coalesce(m.nome, $nome),
      m.sisaweb_id  = coalesce(m.sisaweb_id, $sisaweb_id),
      m.geojson_str = coalesce(m.geojson_str, $geojson_str),
      m.wkt         = coalesce(m.wkt, $wkt)
WITH m, $lon AS lon, $lat AS lat
FOREACH (_ IN CASE WHEN lon IS NULL OR lat IS NULL THEN [] ELSE [1] END |
  SET m.location = point({longitude: lon, latitude: lat})
)
"""

CYPHER_BULK_DIAS = """
UNWIND $dias AS row
MERGE (d:Dia {date: date(row.dt)})
WITH d, row
MATCH (m:Municipio {ibge_id: row.ibge_id})
MERGE (m)-[:TEM_DADO_NO_DIA]->(d)
"""

CYPHER_BULK_METEO = """
UNWIND $met AS row
MATCH (d:Dia {date: date(row.dt)})
MATCH (m:Municipio {ibge_id: row.ibge_id})
MERGE (mm:Meteo {municipio_id: row.ibge_id, date: date(row.dt)})
SET mm += row.meteo_map
MERGE (m)-[:TEM_METEOROLOGIA]->(mm)
MERGE (d)-[:TEM_METEOROLOGIA]->(mm)
"""

CYPHER_BULK_ATIV = """
UNWIND $ativ AS at
MATCH (m:Municipio {ibge_id: at.municipio_id})
MATCH (d:Dia {date: date(at.dt)})
MERGE (a:AtividadeExec {exec_id: at.exec_id})
SET a.municipio_id = at.municipio_id,
    a.date         = date(at.dt),
    a.sisaweb_tipo = at.tipo,
    a.atividade    = at.atividade
SET a += at.props
MERGE (m)-[:EXECUTOU_ATIVIDADE]->(a)
MERGE (a)-[:NO_DIA]->(d)
MERGE (t:TipoAtividade {codigo: at.tipo})
MERGE (a)-[:E_DO_TIPO]->(t)
"""

CYPHER_BULK_ATIV_RESUMO = """
UNWIND $resumos AS r
MATCH (m:Municipio {ibge_id: r.municipio_id})
MATCH (d:Dia {date: date(r.dt)})
MERGE (t:TipoAtividade {codigo: r.tipo})
MERGE (ar:AtividadeResumo {
  municipio_id: r.municipio_id,
  date: date(r.dt),
  tipo: r.tipo
})
SET ar.qtd_execucoes = r.qtd_execucoes
MERGE (m)-[:TEM_ATIVIDADE_RESUMO]->(ar)
MERGE (d)-[:TEM_ATIVIDADE_RESUMO]->(ar)
MERGE (ar)-[:E_DO_TIPO]->(t)
"""

CYPHER_LINK_ATIV_RESUMO = """
UNWIND $resumos AS r
MATCH (ar:AtividadeResumo {
  municipio_id: r.municipio_id,
  date: date(r.dt),
  tipo: r.tipo
})
MATCH (a:AtividadeExec {
  municipio_id: r.municipio_id,
  date: date(r.dt),
  sisaweb_tipo: r.tipo
})
MERGE (a)-[:CONTA_PARA]->(ar)
"""

CYPHER_BULK_CASOS = """
UNWIND $casos AS cs
MATCH (d:Dia {date: date(cs.dt)})
MATCH (m:Municipio {ibge_id: cs.municipio_id})
MERGE (c:Casos {
  municipio_id: cs.municipio_id,
  date: date(cs.dt),
  agravo: cs.agravo
})
SET c += cs.props
MERGE (m)-[:TEM_CASOS]->(c)
MERGE (d)-[:TEM_CASOS]->(c)
MERGE (ag:Agravo {codigo: cs.agravo})
MERGE (c)-[:E_DO_AGRAVO]->(ag)
"""

CYPHER_BULK_NOTIFS = """
UNWIND $notifs AS nf
MATCH (d:Dia {date: date(nf.dt)})
MATCH (m:Municipio {ibge_id: nf.municipio_id})
MERGE (n:Notificacao {notif_id: nf.notif_id})
SET n.municipio_id = nf.municipio_id,
    n.agravo       = nf.agravo,
    n.dt_notific   = CASE 
        WHEN nf.dt_notific IS NULL OR nf.dt_notific = "" OR nf.dt_notific = "        " THEN null 
        ELSE date(nf.dt_notific) 
    END,
    n.id_unidade   = nf.id_unidade,
    n.dt_nasc      = CASE 
        WHEN nf.dt_nasc IS NULL OR nf.dt_nasc = "" OR nf.dt_nasc = "        " THEN null 
        ELSE date(nf.dt_nasc) 
    END,
    n.cs_sexo      = nf.cs_sexo,
    n.cs_gestant   = nf.cs_gestant,
    n.cs_raca      = nf.cs_raca,
    n.cs_escol_n   = nf.cs_escol_n,
    n.dt_obito     = CASE
        WHEN nf.dt_obito IS NULL OR nf.dt_obito = "" OR nf.dt_obito = "        " THEN null
        ELSE date(nf.dt_obito)
    END
SET n += nf.props_adicionais
MERGE (m)-[:TEM_NOTIFICACAO]->(n)
MERGE (d)-[:TEM_NOTIFICACAO]->(n)
MERGE (ag:Agravo {codigo: nf.agravo})
MERGE (n)-[:E_DO_AGRAVO]->(ag)
WITH n, nf
MATCH (c:Casos {
  municipio_id: nf.municipio_id,
  date: date(nf.dt),
  agravo: nf.agravo
})
MERGE (n)-[:CONTA_PARA]->(c)
"""

def close_ring_if_needed(r):
    if not r:
        return r
    if (r[0][0], r[0][1]) != (r[-1][0], r[-1][1]):
        r = r + [r[0]]
    return r

def polygon_to_wkt(coords):
    rings = []
    for ring in coords:
        ring = close_ring_if_needed(ring)
        rings.append("(" + ", ".join(f"{float(x)} {float(y)}" for x, y in ring) + ")")
    return f"POLYGON({','.join(rings)})"

def multipolygon_to_wkt(coords):
    polys = []
    for poly in coords:
        rings = []
        for ring in poly:
            ring = close_ring_if_needed(ring)
            rings.append("(" + ", ".join(f"{float(x)} {float(y)}" for x, y in ring) + ")")
        polys.append("(" + ",".join(rings) + ")")
    return f"MULTIPOLYGON({','.join(polys)})"

def geojson_to_wkt(geo):
    if not isinstance(geo, dict):
        return None
    t = geo.get("type")
    c = geo.get("coordinates")
    if t == "Polygon" and c:
        return polygon_to_wkt(c)
    if t == "MultiPolygon" and c:
        return multipolygon_to_wkt(c)
    return None

def centroid_from_geojson(geo):
    if not isinstance(geo, dict):
        return None
    t = geo.get("type")
    c = geo.get("coordinates")
    ring = c[0] if t == "Polygon" and c else (c[0][0] if t == "MultiPolygon" and c else None)
    if not ring:
        return None
    xs = ys = n = 0.0
    for p in ring:
        if isinstance(p, list) and len(p) >= 2:
            xs += float(p[0])
            ys += float(p[1])
            n += 1
    return (xs / n, ys / n) if n else None

INT64_MIN = -(1 << 63)
INT64_MAX = (1 << 63) - 1

def _safe_number(v):
    if isinstance(v, bool):
        return v
    if isinstance(v, int):
        return v if INT64_MIN <= v <= INT64_MAX else str(v)
    if isinstance(v, float):
        import math
        return v if math.isfinite(v) else str(v)
    return v

def sanitize_value(v):
    if v is None or isinstance(v, (str, bool)):
        return v
    if isinstance(v, (int, float)):
        return _safe_number(v)
    if isinstance(v, list):
        if any(isinstance(i, (list, dict)) for i in v):
            return json.dumps(v, ensure_ascii=False)
        out = []
        for i in v:
            if isinstance(i, (str, bool)) or i is None:
                out.append(i)
            elif isinstance(i, (int, float)):
                out.append(_safe_number(i))
            else:
                return json.dumps(v, ensure_ascii=False)
        return out
    if isinstance(v, dict):
        return json.dumps(v, ensure_ascii=False)
    return json.dumps(v, ensure_ascii=False)

def is_valid_date(date_str):
    if not date_str or not isinstance(date_str, str):
        return False
    date_str = date_str.strip()
    if date_str in ["", "        "]:
        return False
    if len(date_str) == 10 and date_str[4] == "-" and date_str[7] == "-":
        try:
            year = int(date_str[:4])
            month = int(date_str[5:7])
            day = int(date_str[8:10])
            if 1900 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                return True
        except (ValueError, IndexError):
            pass
    return False

def normalize_meteo_dict(m):
    if not isinstance(m, dict):
        return {}
    return {k: sanitize_value(v) for k, v in m.items() if v is not None}

def normalize_casos(doc_ibge, raw):
    casos_resumo, notifs = [], []
    if raw is None:
        return casos_resumo, notifs

    if isinstance(raw, dict):
        for agravo, valor in raw.items():
            if isinstance(valor, list) and all(isinstance(i, dict) for i in valor):
                total, obitos = 0, 0
                for it in valor:
                    dt_notific = it.get("DT_NOTIFIC") or it.get("_DATA") or ""
                    id_unidade = str(it.get("ID_UNIDADE") or "")
                    dt_nasc = str(it.get("DT_NASC") or "")
                    cs_sexo = str(it.get("CS_SEXO") or "")
                    cs_gestant = str(it.get("CS_GESTANT") or "")
                    cs_raca = str(it.get("CS_RACA") or "")
                    cs_escol_n = str(it.get("CS_ESCOL_N") or "")
                    dt_obito = it.get("DT_OBITO") or ""

                    if not is_valid_date(dt_notific):
                        dt_notific = ""
                    if not is_valid_date(dt_nasc):
                        dt_nasc = ""
                    if is_valid_date(dt_obito):
                        obitos += 1
                    else:
                        dt_obito = ""

                    props_adicionais = {}
                    for k, v in it.items():
                        if k in [
                            "DT_NOTIFIC", "_DATA", "ID_UNIDADE", "DT_NASC",
                            "CS_SEXO", "CS_GESTANT", "CS_RACA", "CS_ESCOL_N",
                            "DT_OBITO"
                        ]:
                            continue
                        props_adicionais[k] = sanitize_value(v)
                    
                    total += 1
                    notif_id = f"{doc_ibge}|{agravo}|{dt_notific}|{id_unidade}|{dt_nasc}|{cs_sexo}"
                    
                    notifs.append({
                        "municipio_id": doc_ibge,
                        "agravo": agravo,
                        "notif_id": notif_id,
                        "dt_notific": dt_notific,
                        "id_unidade": id_unidade,
                        "dt_nasc": dt_nasc,
                        "cs_sexo": cs_sexo,
                        "cs_gestant": cs_gestant,
                        "cs_raca": cs_raca,
                        "cs_escol_n": cs_escol_n,
                        "dt_obito": dt_obito,
                        "props_adicionais": props_adicionais
                    })
                
                casos_resumo.append({
                    "municipio_id": doc_ibge,
                    "agravo": agravo,
                    "props": {"qtd": total, "obitos": obitos}
                })
    return casos_resumo, notifs

def normalize_datas(doc):
    datas = []
    dct = doc.get("datas") or {}
    invalid_dates = 0
    
    for dt, dia in dct.items():
        if not is_valid_date(dt):
            invalid_dates += 1
            continue
            
        meteo_map = {}
        if "meteorologia" in dia:
            meteo_map = normalize_meteo_dict(dia["meteorologia"])

        ativ = []
        for k, itens in dia.items():
            if k in ("meteorologia", "casos", "cases", "sinan", "casos_sinan"):
                continue
            if str(k).isdigit():
                itens = itens if isinstance(itens, list) else [itens]
                idx = 0
                for it in itens:
                    raw_props = {}
                    if isinstance(it, dict):
                        raw_props = {kk: sanitize_value(vv) for kk, vv in it.items()}
                    
                    exec_id = f"{doc['ibge_id']}|{dt}|{k}|{idx}"
                    ativ.append({
                        "exec_id": exec_id,
                        "municipio_id": doc["ibge_id"],
                        "tipo": int(k),
                        "atividade": (it.get("atividade") or "").strip() if isinstance(it, dict) else "",
                        "props": raw_props
                    })
                    idx += 1
        
        raw_casos = None
        for alias in ("casos", "cases", "sinan", "casos_sinan"):
            if alias in dia:
                raw_casos = dia[alias]
                break
        
        casos_resumo, casos_notifs = normalize_casos(doc["ibge_id"], raw_casos)
        
        datas.append({
            "dt": dt,
            "meteo_map": meteo_map,
            "ativ": ativ,
            "casos": casos_resumo,
            "notifs": casos_notifs
        })
    
    datas.sort(key=lambda r: r["dt"])
    
    if invalid_dates:
        print(f"  ⚠️  {invalid_dates} datas inválidas ignoradas em {doc.get('ibge_id')}")
    
    return datas

def build_batches(doc):
    geo = doc.get("geo_geometry")
    wkt = geojson_to_wkt(geo) if geo else None
    lon = lat = None
    cent = centroid_from_geojson(geo)
    if cent:
        lon, lat = cent

    datas_norm = normalize_datas(doc)

    dias_params = []
    meteo_params = []
    ativ_params = []
    ativ_resumo_counts = {}
    casos_params = []
    notifs_params = []

    for row in datas_norm:
        dt = row["dt"]
        ibge_id = doc["ibge_id"]

        dias_params.append({"ibge_id": ibge_id, "dt": dt})

        if row["meteo_map"]:
            meteo_params.append({
                "ibge_id": ibge_id,
                "dt": dt,
                "meteo_map": row["meteo_map"]
            })

        for at in row["ativ"]:
            ativ_params.append({
                "exec_id": at["exec_id"],
                "dt": dt,
                "municipio_id": at["municipio_id"],
                "tipo": at["tipo"],
                "atividade": at["atividade"],
                "props": at["props"] or {}
            })
            key = (at["municipio_id"], dt, at["tipo"])
            ativ_resumo_counts[key] = ativ_resumo_counts.get(key, 0) + 1

        for cs in row["casos"]:
            casos_params.append({
                "dt": dt,
                "municipio_id": cs["municipio_id"],
                "agravo": cs["agravo"],
                "props": cs["props"]
            })

        for nf in row["notifs"]:
            notifs_params.append({
                "dt": dt,
                "municipio_id": nf["municipio_id"],
                "agravo": nf["agravo"],
                "notif_id": nf["notif_id"],
                "dt_notific": nf["dt_notific"],
                "id_unidade": nf["id_unidade"],
                "dt_nasc": nf["dt_nasc"],
                "cs_sexo": nf["cs_sexo"],
                "cs_gestant": nf["cs_gestant"],
                "cs_raca": nf["cs_raca"],
                "cs_escol_n": nf["cs_escol_n"],
                "dt_obito": nf["dt_obito"],
                "props_adicionais": nf["props_adicionais"]
            })

    ativ_resumo_params = [
        {
            "municipio_id": mid,
            "dt": dt,
            "tipo": tipo,
            "qtd_execucoes": qtd
        }
        for (mid, dt, tipo), qtd in ativ_resumo_counts.items()
    ]

    return {
        "municipio_payload": {
            "ibge_id": doc["ibge_id"],
            "nome": doc.get("municipio_nome"),
            "sisaweb_id": doc.get("sisaweb_id"),
            "geojson_str": json.dumps(geo, ensure_ascii=False) if geo else None,
            "wkt": wkt,
            "lon": lon,
            "lat": lat
        },
        "dias_params": dias_params,
        "meteo_params": meteo_params,
        "ativ_params": ativ_params,
        "ativ_resumo_params": ativ_resumo_params,
        "casos_params": casos_params,
        "notifs_params": notifs_params
    }

def run_chunked(connection_mgr, query, data_list, param_name, chunk_size=SAFE_CHUNK_SIZE):
    if not data_list:
        return

    with connection_mgr.get_session() as session:
        for i in range(0, len(data_list), chunk_size):
            sub = data_list[i:i + chunk_size]
            if not sub:
                continue
            with session.begin_transaction() as tx:
                tx.run(query, **{param_name: sub})
                tx.commit()

def _total_records_from_batches(b):
    return (
        len(b["dias_params"]) +
        len(b["meteo_params"]) +
        len(b["ativ_params"]) +
        len(b["ativ_resumo_params"]) +
        len(b["casos_params"]) +
        len(b["notifs_params"])
    )

def process_one_file_fast(connection_mgr, doc, prebuilt=None):
    b = prebuilt if prebuilt is not None else build_batches(doc)

    total_records = _total_records_from_batches(b)
    print(f"  ⚡ [FAST] Município {doc['ibge_id']} - {total_records} registros (transação única)")

    with connection_mgr.get_session() as session:
        with session.begin_transaction() as tx:
            tx.run(CYPHER_UPSERT_MUNICIPIO, **b["municipio_payload"])
            if b["dias_params"]:
                tx.run(CYPHER_BULK_DIAS, dias=b["dias_params"])
            if b["meteo_params"]:
                tx.run(CYPHER_BULK_METEO, met=b["meteo_params"])
            if b["ativ_params"]:
                tx.run(CYPHER_BULK_ATIV, ativ=b["ativ_params"])
            if b["ativ_resumo_params"]:
                tx.run(CYPHER_BULK_ATIV_RESUMO, resumos=b["ativ_resumo_params"])
                tx.run(CYPHER_LINK_ATIV_RESUMO, resumos=b["ativ_resumo_params"])
            if b["casos_params"]:
                tx.run(CYPHER_BULK_CASOS, casos=b["casos_params"])
            if b["notifs_params"]:
                tx.run(CYPHER_BULK_NOTIFS, notifs=b["notifs_params"])
            tx.commit()

def process_one_file_chunked(connection_mgr, doc, prebuilt=None):
    b = prebuilt if prebuilt is not None else build_batches(doc)

    total_records = _total_records_from_batches(b)
    print(f"  🛡️ [SAFE] Município {doc['ibge_id']} - {total_records} registros "
          f"(chunked, {SAFE_CHUNK_SIZE} por batch)")

    with connection_mgr.get_session() as session:
        with session.begin_transaction() as tx:
            tx.run(CYPHER_UPSERT_MUNICIPIO, **b["municipio_payload"])
            tx.commit()

    if b["dias_params"]:
        with connection_mgr.get_session() as session:
            with session.begin_transaction() as tx:
                tx.run(CYPHER_BULK_DIAS, dias=b["dias_params"])
                tx.commit()

    if b["meteo_params"]:
        run_chunked(connection_mgr, CYPHER_BULK_METEO, b["meteo_params"], "met")

    if b["ativ_params"]:
        run_chunked(connection_mgr, CYPHER_BULK_ATIV, b["ativ_params"], "ativ")

    if b["ativ_resumo_params"]:
        run_chunked(connection_mgr, CYPHER_BULK_ATIV_RESUMO, b["ativ_resumo_params"], "resumos")
        run_chunked(connection_mgr, CYPHER_LINK_ATIV_RESUMO, b["ativ_resumo_params"], "resumos")

    if b["casos_params"]:
        run_chunked(connection_mgr, CYPHER_BULK_CASOS, b["casos_params"], "casos")

    if b["notifs_params"]:
        run_chunked(connection_mgr, CYPHER_BULK_NOTIFS, b["notifs_params"], "notifs")


def load_progress(path):
    if not os.path.exists(path):
        return set()
    with open(path, "r", encoding="utf-8") as f:
        return {l.strip() for l in f if l.strip()}

def append_progress(path, fname):
    with open(path, "a", encoding="utf-8") as f:
        f.write(fname + "\n")

def main():

    db_manager = Neo4jDatabaseManager(NEO4J_URI, NEO4J_USER, NEO4J_PASS, NEO4J_DB)
    
    if not db_manager.ensure_database_exists():
        return
    
    connection_mgr = Neo4jConnectionManager(NEO4J_URI, NEO4J_USER, NEO4J_PASS, NEO4J_DB)
    
    try:
        with connection_mgr.get_session() as sess:
            for c in CYPHER_CONSTRAINTS:
                try:
                    sess.run(c)
                except ClientError as e:
                    print(f"Aviso constraint: {e}")
                except Exception as e:
                    print(f"Erro ao criar/verificar constraint: {e}")

    except Exception as e:
        connection_mgr.close()
        db_manager.close()
        return

    files = sorted(glob.glob(FOLDER_JSONS))
    if not files:
        raise SystemExit(f"Nenhum JSON em {FOLDER_JSONS}")
    
    done = load_progress(PROGRESS_FILE)
    ok = falha = 0

    for fp in files:
        base = os.path.basename(fp)
        if base in done:
            continue
        

        try:
            with open(fp, "r", encoding="utf-8") as f:
                doc = json.load(f)

            batches = build_batches(doc)
            total_records = _total_records_from_batches(batches)

            sucesso = False

            if total_records <= MAX_FAST_RECORDS:
                try:
                    process_one_file_fast(connection_mgr, doc, prebuilt=batches)
                    sucesso = True
                except Exception as e_fast:
                    try:
                        process_one_file_chunked(connection_mgr, doc, prebuilt=batches)
                        sucesso = True
                    except Exception as e_safe:
                        sucesso = False
            else:
                try:
                    process_one_file_chunked(connection_mgr, doc, prebuilt=batches)
                    sucesso = True
                except Exception as e_safe:
                    sucesso = False
            
            if sucesso:
                append_progress(PROGRESS_FILE, base)
                ok += 1
            else:
                falha += 1

        except Exception as e_outer:
            falha += 1

    try:
        with connection_mgr.get_session() as sess:
            r1 = sess.run("MATCH (m:Municipio) RETURN count(m) AS n").single()["n"]
            r2 = sess.run("MATCH (d:Dia) RETURN count(d) AS n").single()["n"]
            r3 = sess.run("MATCH (c:Casos) RETURN count(c) AS n").single()["n"]
            r4 = sess.run("MATCH (n:Notificacao) RETURN count(n) AS n").single()["n"]
            rA = sess.run("MATCH (a:AtividadeExec) RETURN count(a) AS n").single()["n"]
            rT = sess.run("MATCH (t:TipoAtividade) RETURN count(t) AS n").single()["n"]
            rG = sess.run("MATCH (g:Agravo) RETURN count(g) AS n").single()["n"]
            rAR = sess.run("MATCH (ar:AtividadeResumo) RETURN count(ar) AS n").single()["n"]

    except Exception as e:
        print(f"Erro ao obter métricas finais: {e}")

    connection_mgr.close()
    db_manager.close()

if __name__ == "__main__":
    main()
